In [8]:
import pandas as pd
dataFile='C:\\Users\\Alexander\\Python\\olmedo-vinueza-alexander-daniel\\06-Recomendation-Algorithm\\data\\BX-Book-Ratings.csv'


In [11]:
data=pd.read_csv(dataFile,sep=";",header=0,names=["user","isbn","rating"], encoding='iso-8859-1' )

In [13]:
data.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [15]:
archivo_books='C:\\Users\\Alexander\\Python\\olmedo-vinueza-alexander-daniel\\06-Recomendation-Algorithm\\data\\BX-Books.csv'
books=pd.read_csv(
    archivo_books,sep=";",
    header=0,
    error_bad_lines=False, 
    usecols=[0,1,2],
    index_col=0,names=['isbn',"title","author"],
    encoding='iso-8859-1')


In [16]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [17]:
def book_titulo_autor(isbn):
    titulo = books.at[isbn,"title"]
    autor = books.at[isbn,"author"]
    return titulo, autor

In [18]:
book_titulo_autor('0002005018')

('Clara Callan', 'Richard Bruce Wright')

In [23]:
def n_libros_favoritos_por_usuario(usuario,N):
    ratings_usuario = data[data["user"]==usuario]  # Filtering by user  (relevant data)
    ratings_usuario_ordenado = pd.DataFrame.sort_values(
        ratings_usuario,
        ['rating'],
        ascending=[0])[:N]  # Sort by rating in desc order pick N
    ratings_usuario_ordenado["title"] = ratings_usuario_ordenado["isbn"].apply(book_titulo_autor)  # Apply bookMeta to ISBN column
    return ratings_usuario_ordenado

In [26]:
data = data[data["isbn"].isin(books.index)]  # Inconsistencies


In [29]:
n_libros_favoritos_por_usuario(276729,5)

,user,isbn,rating,title
4,276729,0521795028,6,(The Amsterdam Connection : Level 4 (Cambridge...
3,276729,052165615X,3,"(Help!: Level 1, Philip Prowse)"


In [30]:
data.shape  # more than 1 million

(1031175, 3)

In [35]:
ratings_por_libro = data.isbn.value_counts()
ratings_por_libro.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [33]:
ratings_usuarios_por_isbn.shape

(270170,)

In [36]:
ratings_por_usuario = data.user.value_counts()
ratings_por_usuario.head(10)

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
212898     4290
278418     3996
76352      3329
110973     2971
235105     2943
Name: user, dtype: int64

In [37]:
ratings_por_usuario.shape

(92107,)

In [38]:
data = data[data["isbn"].isin(ratings_por_libro[ratings_por_libro>10].index)]  # ISBN read for more than 10 users

In [39]:
data = data[data["user"].isin(ratings_por_usuario[ratings_por_usuario>10].index)]  # ISBN users read more than 10 books

In [41]:
matriz_ratings_por_usuario=pd.pivot_table(data, 
                                          values='rating',
                                          index=['user'],
                                          columns=['isbn'])  # Rating Matrix (pivot)

In [45]:
matriz_ratings_por_usuario.shape

(10706, 15451)

In [46]:
usuario_1 = 204622
usuario_2 = 255489

In [47]:
usuario_1_ratings = matriz_ratings_por_usuario.transpose()[usuario_1]  # User 1
usuario_1_ratings.head()

isbn
0002005018   NaN
0002251760   NaN
0002259834   NaN
0002558122   NaN
0006480764   NaN
Name: 204622, dtype: float64

In [48]:
usuario_2_ratings = matriz_ratings_por_usuario.transpose()[usuario_2]  # User 2
usuario_2_ratings.head()

isbn
0002005018   NaN
0002251760   NaN
0002259834   NaN
0002558122   NaN
0006480764   NaN
Name: 255489, dtype: float64

In [49]:
## Compute de hamming distance

from scipy.spatial.distance import hamming 
hamming(usuario_1_ratings,usuario_2_ratings)  # Hamming distance

0.9999352792699502

In [50]:
import numpy as np
# Distance by any pair of users
def distancia_entre_dos_usuarios(usuario_1,usuario_2):
        try:
            ratings_usuario_1 = matriz_ratings_por_usuario.transpose()[usuario_1]
            ratings_usuario_2 = matriz_ratings_por_usuario.transpose()[usuario_2]
            distancia = hamming(ratings_usuario_1,ratings_usuario_2)
        except: 
            distancia = np.NaN
        return distancia 

In [53]:
distancia_entre_dos_usuarios(usuario_1,usuario_2)

0.9999352792699502

In [57]:
usuario_a_recomendar = 204622
todos_usuarios = pd.DataFrame(matriz_ratings_por_usuario.index)
todos_usuarios = todos_usuarios[todos_usuarios.user!=usuario_a_recomendar]  # Remove the actual user to recommend
todos_usuarios.head()

,user
0,8
1,99
2,242
3,243
4,254


In [62]:
todos_usuarios["distance"] = todos_usuarios["user"].apply(
    lambda usuario_fila_actual: distancia_entre_dos_usuarios(usuario_a_recomendar,usuario_fila_actual))  # Apply the distance function to this new column

In [63]:
todos_usuarios

,user,distance
0,8,1.000000
1,99,1.000000
2,242,0.999935
3,243,0.999935
4,254,1.000000
5,383,1.000000
6,388,1.000000
7,408,1.000000
8,424,1.000000
9,446,1.000000


In [64]:
K = 10
K_valores_mas_cercanos = todos_usuarios.sort_values(["distance"],ascending=True)["user"][:K]  # Find the K nearest neighbors

In [65]:
K_valores_mas_cercanos

3201     82893
3368     87555
2624     68555
1813     48046
5401    140036
7584    198711
565      16795
8866    232131
239       7346
9693    251422
Name: user, dtype: int64

In [66]:
# Make the function, sending user and number of K users
def nearest_neighbors(usuario,K=10):
    todos_usuarios = pd.DataFrame(matriz_ratings_por_usuario.index)
    todos_usuarios = todos_usuarios[todos_usuarios.user!=usuario]
    todos_usuarios["distance"] = todos_usuarios["user"].apply(
        lambda usuario_fila_actual: distancia_entre_dos_usuarios(usuario,usuario_fila_actual))
    K_usuarios_mas_cercanos = todos_usuarios.sort_values(
        ["distance"],ascending=True)["user"][:K]
    return K_usuarios_mas_cercanos

In [68]:
nearest_neighbors(usuario_a_recomendar, 5)


3201     82893
3368     87555
2624     68555
1813     48046
5401    140036
Name: user, dtype: int64

In [69]:
K_usuarios_mas_cercanos = nearest_neighbors(usuario_a_recomendar)
K_usuarios_mas_cercanos

3201     82893
3368     87555
2624     68555
1813     48046
5401    140036
7584    198711
565      16795
8866    232131
239       7346
9693    251422
Name: user, dtype: int64

In [70]:
N_Ratings = matriz_ratings_por_usuario[matriz_ratings_por_usuario.index.isin(K_usuarios_mas_cercanos)]  # Ratings of the Nearest Neighbors for all books
N_Ratings

isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
7346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
avg_rating = N_Ratings.apply(np.nanmean).dropna()  # Average rating per isbn, nanmean is applied for each column (isbn) dropna() drop books which do not have rating
avg_rating.head()

isbn
0007154615    1.5
0020125305    0.0
0020125607    0.0
0020198817    0.0
0020198906    8.0
dtype: float64

In [79]:
libros_ya_leidos = matriz_ratings_por_usuario.transpose()[usuario_a_recomendar].dropna().index  # Get ratings of the active user, drop books without rating
libros_ya_leidos

Index(['006016848X', '0060935464', '0140042598', '0140178724', '0142004278',
       '0380732238', '0385504209', '0425109720', '0425152898', '0440136482',
       '0440241162', '0451191145', '0451197127', '0553096060', '0671027360',
       '0671027387', '0671666258', '0688174574', '0743225708', '076790592X',
       '0785264280', '0786868716', '0802131867', '0802132952', '0971880107',
       '1853260045', '1853260126', '1853260207', '185326041X', '1878424114'],
      dtype='object', name='isbn')

In [80]:
avg_rating = avg_rating[~avg_rating.index.isin(libros_ya_leidos)]  # Remove average of already read books

In [81]:
N=3
topNISBNs = avg_rating.sort_values(ascending=False).index[:N]  # Sort values and pick N

In [82]:
pd.Series(topNISBNs).apply(book_titulo_autor)  # Tell us title and author

0              (Love, Greg &amp; Lauren, Greg Manning)
1    (The Two Towers (The Lord of the Rings, Part 2...
2    (Harry Potter and the Sorcerer's Stone (Book 1...
Name: isbn, dtype: object

In [85]:
# function for any user

def topN(usuario,N=3):
    K_usuarios_mas_cercanos = nearest_neighbors(usuario)
    N_Ratings = matriz_ratings_por_usuario[matriz_ratings_por_usuario.index.isin(K_usuarios_mas_cercanos)]
    avg_rating = N_Ratings.apply(np.nanmean).dropna()
    libros_ya_leidos = matriz_ratings_por_usuario.transpose()[usuario].dropna().index
    avg_rating = avg_rating[~avg_rating.index.isin(libros_ya_leidos)]
    top_N_libros = avg_rating.sort_values(ascending=False).index[:N]
    return pd.Series(top_N_libros).apply(book_titulo_autor)
    


0    (Waiting For Nick (Silhouette Special Edition)...
1           (Wringer (Trophy Newbery), Jerry Spinelli)
2    (The Star Wars Trilogy: Star Wars, the Empire ...
3          (One, Two, Buckle My Shoe, Agatha Christie)
4                          (On the Road, Jack Kerouac)
5                 (Dead Poets Society, N.H. Kleinbaum)
6     (Go Ask Alice (Avon/Flare Book), James Jennings)
7                        (Carolina Moon, Nora Roberts)
8    (Illusions: The Adventures of a Reluctant Mess...
9    (You Just Don't Duct Tape a Baby!: True Tales ...
Name: isbn, dtype: object

In [86]:
topN(usuario_a_recomendar,10)

0              (Love, Greg &amp; Lauren, Greg Manning)
1    (The Two Towers (The Lord of the Rings, Part 2...
2    (Harry Potter and the Sorcerer's Stone (Book 1...
3                       (Charlotte's Web, E. B. White)
4             (The Secret Life of Bees, Sue Monk Kidd)
5    (My Sister's Keeper : A Novel (Picoult, Jodi),...
6    (Small Sacrifices: A True Story of Passion and...
7    (Snake, the Crocodile &amp; the Dog, The (Amel...
8    (All Things Bright and Beautiful (All Things B...
9               (The Liar's Club: A Memoir, Mary Karr)
Name: isbn, dtype: object